<a href="https://colab.research.google.com/github/kasri-mids/MovieGenreClassification/blob/master/src/Data/3_Split_Poster_Feature_Vectors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Splitting Poster Feature Vectors for train, dev and val

In this notebook, we take our cleaned dataset and split the feature vectors for the movie posters into a train, dev and validation set and we save those files for later use.

We do this for both, 1 label to perform baseline error analysis as well as 10 labels for the entire model.

In [0]:
# Imports

import json
import pandas as pd
import numpy as np
import os
import sys
import tensorflow as tf
import tensorflow_hub as hub
from time import time
import io
import re
from csv import reader
from datetime import datetime
import matplotlib.pyplot as plt

import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.ticker import PercentFormatter

from tensorflow.keras import backend as K
from tensorflow.keras import layers
from tensorflow.python.keras.layers import Lambda
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.backend import sparse_categorical_crossentropy
from tensorflow.keras.layers import Dense, TimeDistributed

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# ERROR_TEST = 1
ERROR_TEST = 0

In [0]:
if (ERROR_TEST):
    use_cols = ['movieid', 'movieplot', 'labels']
    df = pd.read_csv('/content/drive/My Drive/w266-FinalProject/MoviePlotsAndPosters/data/clean_poster_data_1.csv',
                     usecols=use_cols)    
    NUM_LABELS = 1
    data_save_prefix = '/content/drive/My Drive/w266-FinalProject/MoviePlotsAndPosters/data/baseline_test/'
    model_save_prefix = '/content/drive/My Drive/w266-FinalProject/MoviePlotsAndPosters/model/baseline_test/'
    plot_save_prefix = '/content/drive/My Drive/w266-FinalProject/MoviePlotsAndPosters/plots/baseline_test/'
else:
    use_cols = ['movieid', 'movieplot', 'labels']
    df = pd.read_csv('/content/drive/My Drive/w266-FinalProject/MoviePlotsAndPosters/data/clean_poster_data_10.csv',
                     usecols=use_cols)
    NUM_LABELS = 10
    data_save_prefix = '/content/drive/My Drive/w266-FinalProject/MoviePlotsAndPosters/data/baseline/'
    model_save_prefix = '/content/drive/My Drive/w266-FinalProject/MoviePlotsAndPosters/model/baseline/'
    plot_save_prefix = '/content/drive/My Drive/w266-FinalProject/MoviePlotsAndPosters/plots/baseline/'

In [0]:
df.movieid = df.movieid + '.jpg'
df.head()

,movieid,movieplot,labels
0,tt0007610.jpg,A judge who had taken part in the gold rush of...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,tt0005078.jpg,"Richard Hardy, a hardworking stockbroker, labo...","[1, 0, 0, 0, 0, 0, 1, 0, 0, 0]"
2,tt0007257.jpg,"Reggie, a wealthy young man about town, is eag...","[1, 0, 1, 1, 0, 0, 0, 0, 0, 0]"
3,tt0008011.jpg,"Living in the country with her widowed mother,...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]"
4,tt0003772.jpg,Based on the fairy tale: Cinderella is mistrea...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [0]:
df.shape

(78008, 3)

In [0]:
poster_feature_vector = np.load('/content/drive/My Drive/w266-FinalProject/data/posters/image_feature_vectors.data.npy',allow_pickle=True)
poster_feature_vector.shape

(81124, 2048)

In [0]:
poster_tracking_id = pd.read_csv('/content/drive/My Drive/w266-FinalProject/data/posters/image_tracking_id.data', header=None)
poster_tracking_id.shape

(81124, 1)

In [0]:
poster_tracking_id.loc[:,'indexes'] = poster_tracking_id.index

In [0]:
poster_tracking_id.columns = ['movieid_poster', 'indexes']

In [0]:
poster_tracking_id.head()

,movieid_poster,indexes
0,tt1062961.jpg,0
1,tt3885736.jpg,1
2,tt0027902.jpg,2
3,tt5066056.jpg,3
4,tt0052306.jpg,4


In [0]:
# Merge df with posters to get 78008 rows
df_merge = poster_tracking_id.merge(df, how='inner', left_on='movieid_poster', right_on='movieid')

In [0]:
df_merge.shape

(78008, 5)

In [0]:
df_merge.head()

,movieid_poster,indexes,movieid,movieplot,labels
0,tt1062961.jpg,0,tt1062961.jpg,"A young photographer moves to Praga, the most ...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]"
1,tt0027902.jpg,2,tt0027902.jpg,"As a young lad, Jonathan Blake and his friend ...","[1, 0, 0, 0, 0, 0, 1, 0, 0, 0]"
2,tt5066056.jpg,3,tt5066056.jpg,A retired film star's wedding to her fourth hu...,"[0, 0, 1, 0, 0, 0, 1, 0, 0, 0]"
3,tt0052306.jpg,4,tt0052306.jpg,"Duclos's son, a worldly gangster, is compromis...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]"
4,tt0064390.jpg,5,tt0064390.jpg,Twenty-five year old Pierre is a man who likes...,"[0, 0, 1, 0, 0, 0, 1, 0, 0, 0]"


In [0]:
# We need to take out poster feature vectors only for the indexes in the above df_merge
merged_poster_feature_vector = poster_feature_vector[df_merge.indexes.values]

In [0]:
len(merged_poster_feature_vector)

78008

In [0]:
# Saves poster feature vectors
np.save('/content/drive/My Drive/w266-FinalProject/MoviePlotsAndPosters/data/clean_poster_feature_vector', merged_poster_feature_vector)

# Save relevant 78k movieids
df_merge[['movieid']].to_csv('/content/drive/My Drive/w266-FinalProject/MoviePlotsAndPosters/data/clean_poster_movieid_data.csv', index=False)

In [0]:
import ast
labels = [ast.literal_eval(x) for x in df_merge.labels]
len(labels)

78008

In [0]:
labels[:5]

[[0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 1, 0, 0, 0]]

In [0]:
# Save relevant labels
np.save('/content/drive/My Drive/w266-FinalProject/MoviePlotsAndPosters/data/clean_poster_labels_' + str(NUM_LABELS), labels)

In [0]:
# Next, split poster vectors and labels into train, test and val

In [0]:
def split_data(numExamples, listInputs, fraction):
    np.random.seed(0)
    training_examples = np.random.binomial(1, fraction, numExamples)
    # print("Inside Training Examples ", training_examples[:20])

    indexArray = np.arange(0,numExamples,1,dtype=np.int32)

    np.random.seed(0)
    np.random.shuffle(indexArray)
    # print("Inside Index Array ", indexArray[:20])

    list_1 = []
    list_2 = []

    for example in indexArray:
        if training_examples[example] == 1:
            list_1.append((listInputs[example]))
        else:
            list_2.append((listInputs[example]))
    
    list_1 = np.array(list_1)
    list_2 = np.array(list_2)

    return (list_1, list_2, indexArray, training_examples)

In [0]:
numExamples = df_merge.shape[0]
posterVectors_train, posterVectors_test, indexArray, training_examples = split_data(numExamples, merged_poster_feature_vector, 0.6)
genreLabels_train, genreLabels_test, indexArray, training_examples = split_data(numExamples, labels, 0.6)

In [0]:
train_indexes = []
test_indexes = []
for index in indexArray:
    if training_examples[index] == 1:
        train_indexes.append(index)
    else:
        test_indexes.append(index)

In [0]:
df_train = df_merge.iloc[train_indexes]
df_temp = df_merge.iloc[test_indexes]

In [0]:
numExamples = posterVectors_test.shape[0]
posterVectors_dev, posterVectors_val, indexArray, training_examples = split_data(numExamples, posterVectors_test, 0.5)
genreLabels_dev, genreLabels_val, indexArray, training_examples = split_data(numExamples, genreLabels_test, 0.5)

In [0]:
dev_indexes = []
val_indexes = []
for index in indexArray:
    if training_examples[index] == 1:
        dev_indexes.append(index)
    else:
        val_indexes.append(index)

In [0]:
df_dev = df_temp.iloc[dev_indexes]
df_val = df_temp.iloc[val_indexes]

In [0]:
print(posterVectors_train.shape)
print(posterVectors_dev.shape)
print(posterVectors_val.shape)

(46970, 2048)
(15438, 2048)
(15600, 2048)


In [0]:
print(genreLabels_train.shape)
print(genreLabels_dev.shape)
print(genreLabels_val.shape)

(46970, 10)
(15438, 10)
(15600, 10)


In [0]:
print(df_train.shape)
print(df_dev.shape)
print(df_val.shape)

(46970, 5)
(15438, 5)
(15600, 5)


In [0]:
np.save('/content/drive/My Drive/w266-FinalProject/MoviePlotsAndPosters/data/clean_train_poster_vectors.data', posterVectors_train)
np.save('/content/drive/My Drive/w266-FinalProject/MoviePlotsAndPosters/data/clean_dev_poster_vectors.data', posterVectors_dev)
np.save('/content/drive/My Drive/w266-FinalProject/MoviePlotsAndPosters/data/clean_val_poster_vectors.data', posterVectors_val)

In [0]:
np.save('/content/drive/My Drive/w266-FinalProject/MoviePlotsAndPosters/data/clean_train_poster_labels_' + str(NUM_LABELS) + '.data', genreLabels_train)
np.save('/content/drive/My Drive/w266-FinalProject/MoviePlotsAndPosters/data/clean_dev_poster_labels_' + str(NUM_LABELS) + '.data', genreLabels_dev)
np.save('/content/drive/My Drive/w266-FinalProject/MoviePlotsAndPosters/data/clean_val_poster_labels_' + str(NUM_LABELS) + '.data', genreLabels_val)

In [0]:
# Save this, same regardless of labels
df_train.to_csv('/content/drive/My Drive/w266-FinalProject/MoviePlotsAndPosters/data/clean_df_train_labels_' + str(NUM_LABELS) + '.csv', index=False)
df_dev.to_csv('/content/drive/My Drive/w266-FinalProject/MoviePlotsAndPosters/data/clean_df_dev_labels_' + str(NUM_LABELS) + '.csv', index=False)
df_val.to_csv('/content/drive/My Drive/w266-FinalProject/MoviePlotsAndPosters/data/clean_df_val_labels_' + str(NUM_LABELS) + '.csv', index=False)